In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [3]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [6]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [15]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [170]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [20]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [90]:
# load link counts data
link_day_Jan_dict = zload('../temp_files/link_day_Jan_dict.pkz')
link_day_Apr_dict = zload('../temp_files/link_day_Apr_dict.pkz')
link_day_Jul_dict = zload('../temp_files/link_day_Jul_dict.pkz')
link_day_Oct_dict = zload('../temp_files/link_day_Oct_dict.pkz')

In [296]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [297]:
K = len(week_day_Jan_list) + len(week_day_Apr_list) + len(week_day_Jul_list) + len(week_day_Oct_list)
# K = len(week_day_Jan_list)

In [298]:
K

88

In [299]:
x = np.zeros((24, K))

In [300]:
m = 24

for k_1 in range(len(week_day_Jan_list)):
    for i in range(m):
        x[i, k_1] = link_day_Jan_dict['link_' + str(i) + '_' + str(week_day_Jan_list[k_1])].AM_flow

for k_2 in range(len(week_day_Apr_list)):
    for i in range(m):
        x[i, k_2 + k_1 + 1] = link_day_Apr_dict['link_' + str(i) + '_' + str(week_day_Apr_list[k_2])].AM_flow
for k_3 in range(len(week_day_Jul_list)):
    for i in range(m):
        x[i, k_3 + k_2 + k_1 + 2] = link_day_Jul_dict['link_' + str(i) + '_' + str(week_day_Jul_list[k_3])].AM_flow
for k_4 in range(len(week_day_Oct_list)):
    for i in range(m):
        x[i, k_4 + k_3 + k_2 + k_1 + 3] = link_day_Oct_dict['link_' + str(i) + '_' + str(week_day_Oct_list[k_4])].AM_flow
        
x = np.matrix(x)
x = np.nan_to_num(x)

In [322]:
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [287]:
np.size(x, 1)

86

In [288]:
link_day_Jan_dict['link_2_2'].AM_flow, link_day_Jan_dict['link_2_1'].AM_flow

(639708.01714969077, nan)

In [289]:
np.size(x,0), np.size(x,1)

(24, 86)

In [290]:
np.size(A,0), np.size(A,1)

(24, 140)

In [301]:
# calculate the GLS solution for the O-D matrix

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

term_1 = inv(np.dot(np.dot(PA_t, inv_S), AP_t))
term_2 = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])

lam_est = (1.0 / K) * np.dot(term_1, term_2)

In [323]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-03, 2e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.72130838e+06  0.00000000e+00  6.73e+03 3.76e+02  8.98e+05     0s
   1   8.64381920e+05 -2.62020322e+06  1.36e+03 2.00e+02  2.43e+05     0s
   2   4.62314811e+05 -1.58149847e+06  1.36e-03 1.14e-13  3.65e+04     0s
   3  -2.36114727e+05 -5.09350639e+05  1.36e-09 5.68e-14  4.8

In [317]:
lam

[<gurobi.Var lam_0 (value 44108.5679967)>,
 <gurobi.Var lam_1 (value 77575.6030945)>,
 <gurobi.Var lam_2 (value 102683.512651)>,
 <gurobi.Var lam_3 (value 6.14509203991e-08)>,
 <gurobi.Var lam_4 (value 0.0333553961187)>,
 <gurobi.Var lam_5 (value 12.6964943976)>,
 <gurobi.Var lam_6 (value 67090.195218)>,
 <gurobi.Var lam_7 (value 783754.986485)>,
 <gurobi.Var lam_8 (value 2.27004710245e-09)>,
 <gurobi.Var lam_9 (value 8.61992711646e-09)>,
 <gurobi.Var lam_10 (value 2.18917713316e-09)>,
 <gurobi.Var lam_11 (value 2.27004705365e-09)>,
 <gurobi.Var lam_12 (value 2.27004709844e-09)>,
 <gurobi.Var lam_13 (value 2.52200436345e-09)>,
 <gurobi.Var lam_14 (value 4.01873835162e-07)>,
 <gurobi.Var lam_15 (value 1.05725011598e-07)>,
 <gurobi.Var lam_16 (value 5.02813083138e-09)>,
 <gurobi.Var lam_17 (value 6.14509151127e-08)>,
 <gurobi.Var lam_18 (value 206949.8737)>,
 <gurobi.Var lam_19 (value 123227.461201)>,
 <gurobi.Var lam_20 (value 5.65722952508e-08)>,
 <gurobi.Var lam_21 (value 64779.019512

In [320]:
len(x[:,1])

24

In [321]:
K

88